<a href="https://colab.research.google.com/github/Ajjme/Climate_system_modeling/blob/main/Energy_Demand_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Constant

In [ ]:
#equations

In [4]:
#this is Energy Consuming Capital
# I removed the vintage things as I dont understand them


class EnergyConsumingCapital1:
    def __init__(self, end_use_sectors, initial_capital, gdp_by_sector, capital_lifetime):
        """
        Initializes the model for energy-consuming capital in various end-use sectors.

        :param end_use_sectors: List of end-use sectors.
        :param initial_capital: Dictionary with initial capital for each sector.
        :param gdp_by_sector: Dictionary with GDP values for each sector.
        :param capital_lifetime: Dictionary with capital lifetime for each sector.
        """

        self.end_use_sectors = end_use_sectors
        self.capital = {sector: initial_capital.get(sector, 0) for sector in end_use_sectors} #get initial values that are inputs
        self.gdp_by_sector = gdp_by_sector
        self.capital_lifetime = capital_lifetime
        self.initial_capital_output_ratio = 2.5 #given in Enroads
        self.time_to_adjust = 1 #given in Enroads
        self.discarding_rate = {sector: min(1, max(0, 1 / (capital_lifetime[sector] / 3))) for sector in end_use_sectors}

    def desired_capital(self, sector):
        """Calculates the desired capital for a given sector."""
        return self.gdp_by_sector.get(sector, 0) * self.initial_capital_output_ratio #just gdp times intial capital output ratio given as 2.5

    def installing_capital(self, sector, total_energy_capital, discarded_retired_capital):
        """Calculates the new capital installations for a given sector."""
        return max(0, (self.desired_capital(sector) - total_energy_capital) / (self.time_to_adjust + discarded_retired_capital)) #max 0 or this number

    def discarding_capital_1(self, sector):
        """Calculates the discarding of capital in phase 1 due to cost increases."""
        return self.capital[sector] * self.discarding_rate[sector] #

    def retiring_capital(self, sector):
        """Calculates the discarding of capital in phase 1 due to cost increases."""
        return self.capital[sector] * self.discarding_rate[sector] #

    def discarded_and_retired_capital(self, sector, retiring):
        """Calculates the total discarded and retired capital for a given sector."""
        return retiring + self.discarding_capital_1(sector)  #retiring is a given...

    def update_capital(self, sector, retiring, total_energy_capital):
        """Updates the energy-consuming capital for a given sector."""
        discarded_retired = self.discarded_and_retired_capital(sector, retiring)
        new_installations = self.installing_capital(sector, total_energy_capital, discarded_retired)
        self.capital[sector] += new_installations - discarded_retired

    def get_capital(self, sector):
        """Returns the current energy-consuming capital for a given sector."""
        return self.capital.get(sector, 0)

# Example Usage
end_use_sectors = ["Residential and Commercial", "Industrial", "Transport"] #the three groups
initial_capital = {"Residential and Commercial": 57, "Industrial": 50, "Transport": 20} #values from straight line in Enroads
gdp_by_sector = {"Residential and Commercial": 1000, "Industrial": 1500, "Transport": 20} #this comes from another module need to update #todo
capital_lifetime = {"Residential and Commercial": 30, "Industrial": 30,"Transport": 15} #straight lines in Enroads #each vintage has a different life time

model = EnergyConsumingCapital1(end_use_sectors, initial_capital, gdp_by_sector, capital_lifetime)

# Simulating an update
model.update_capital("Residential and Commercial", retiring=5, total_energy_capital=0) #retiring needs to be added
#total_energy_capital set randomly (do they want the value from each enduse so need to loop in each one?)
#(Total_energy_Capital loops on all others? Total energy consuming capital for each end use[EndUseSector]= Energy consuming capital 1[EndUseSector] + Energy consuming capital 2[EndUseSector]+ Energy consuming capital 3[EndUseSector]

print(model.get_capital("Residential and Commercial"))  # Updated capital value need to give they system what sector you want can change that in get_capital function


259.9752136752137


In [ ]:
import math

#this equation needs to be able to work for all Carriers(coal...)

class ComplementaryAssets:
    def __init__(self, initial_assets, installation_capacity, k_soft, f_soft, v_soft):
        """
        Initialize the ComplementaryAssets class.

        Parameters:
        initial_assets (float): Initial complementary assets ($)
        installation_capacity (float): Complementary asset installation capacity ($/Year)
        k_soft (float): Soft limit parameter
        f_soft (float): Soft limit parameter
        v_soft (float): Soft limit parameter
        """
        self.complementary_assets = initial_assets
        self.installation_capacity = installation_capacity
        self.k_soft = k_soft
        self.f_soft = f_soft
        self.v_soft = v_soft

    def increase_in_complementary_assets(self, indicated_increase):
        """
        Calculate the annual increase in complementary assets using a fuzzy MIN function.

        Parameters:
        indicated_increase (float): Indicated increase in complementary assets ($/Year)

        Returns:
        float: Increase in complementary assets ($/Year)
        """
        if self.installation_capacity <= 0:
            return 0

        fuzzy_min = (1 / (1 + self.k_soft * math.exp(-self.f_soft * (indicated_increase / self.installation_capacity)))) ** self.v_soft

        return max(0, self.installation_capacity * fuzzy_min)

    def update_complementary_assets(self, indicated_increase, time_step):
        """
        Update the complementary assets over time.

        Parameters:
        indicated_increase (float): Indicated increase in complementary assets ($/Year)
        time_step (float): Time step for integration (Years)
        """
        increase = self.increase_in_complementary_assets(indicated_increase)
        self.complementary_assets += increase * time_step

# Example usage
initial_assets = 10000
installation_capacity = 5000

k_soft = 2 #from Enroads
f_soft = 4 #from Enroads
v_soft = 3 #from Enroads
time_step = 1  # 1 year

assets_model = ComplementaryAssets(initial_assets, installation_capacity, k_soft, f_soft, v_soft)
indicated_increase = 1000

assets_model.update_complementary_assets(indicated_increase, time_step)
print("Updated Complementary Assets ($):", assets_model.complementary_assets)


In [ ]:
#Cumulative carrier installed
#units EJ/Year
#equations
#Cumulative carrier installed[EndUseSector,Carrier]
 #= #INTEG( Energy requirements installing[Carrier, EndUseSector],
#Initial cumulative carrier installed[Carrier, EndUseSector] )

**Conceptual Meaning**
This equation measures how far the current tech effect rate is from the expected energy efficiency improvements.
Before the start year (e.g., 2025), it assumes a reference improvement rate. After that, it switches to a different expected improvement.
The result is a gap that indicates how much improvement is still needed to reach the expected energy efficiency levels.

The TechEffectModel should output the **Tech effect rate relative to target** by integrating the change in tech over time. This is done by multiplying the tech effect rate adjustment for each end use by the tech effect rat ratio of each enduse.
The tech adjustment rate is calculated by dividing the Tech effect rate gap by the time it takes to reach the long term intensity rate

The tech effect rate Gap is The gap between the GDP/capita rate target and actual, as a fraction of the actual calculated by selecting one of the improvement rate inputs based on year that number is divided by the rate of decreac in a current year minus the tech effect rate ratio which is the gap beterrn the expected improvement and actural

**Output**
This code give a list of year and relative Tech Gap rate by sector (Transportation and stationary)

**Future Use**
this gets plugged into...

In [ ]:
class TechEffectModel:
    def __init__(self, sector, start_year, ref_annual_improvement, annual_improvement, ei_improvement_rate, time_to_reach_long_term_rate):
        self.sector = sector
        self.start_year = start_year
        self.ref_annual_improvement = ref_annual_improvement
        self.annual_improvement = annual_improvement
        self.ei_improvement_rate = ei_improvement_rate
        self.time_to_reach_long_term_rate = time_to_reach_long_term_rate
        self.tech_effect_rate_ratio = 1  # Initial condition

    def zidz(self, numerator, denominator):
        return numerator / denominator if denominator != 0 else 0 #(a function from endroads that says the answer is zero if divided by zero)

    def rate_of_decrease(self, time):
        return self.ei_improvement_rate #This equation determines the rate at which energy intensity (energy consumption per unit of output) decreases over time. #seems like it just returns the same thing no matter what But it hink we will adjust this value in other sections of the module making this dynamic

    def tech_effect_rate_gap(self, time):
        improvement_rate = self.ref_annual_improvement if time < self.start_year else self.annual_improvement
        return self.zidz(improvement_rate / self.rate_of_decrease(time) - self.tech_effect_rate_ratio, self.tech_effect_rate_ratio) #this is the Heart

    def tech_effect_rate_adjustment(self, time):
        return self.tech_effect_rate_gap(time) / self.time_to_reach_long_term_rate #Tech effect rate adjustment[EndUseSector] = Tech effect rate Gap[EndUseSector] / Time to reach long term intensity rate

    def change_in_tech_effect_rate(self, time):
        if time <= 2021:
            return 0
        return self.tech_effect_rate_adjustment(time) * self.tech_effect_rate_ratio #IF THEN ELSE( Time <= y2021, 0, Tech effect rate adjustment[EndUseSector] * Tech effect rate ratio[EndUseSector] )

    def update_tech_effect_rate_ratio(self, time):
        self.tech_effect_rate_ratio += self.change_in_tech_effect_rate(time)

    def simulate(self, time_steps):
        results = []
        for time in time_steps:
            self.update_tech_effect_rate_ratio(time)
            results.append((time, self.tech_effect_rate_ratio))
        return results

# Example Usage
if __name__ == "__main__":
    stationary_sector = TechEffectModel(
        sector="stationary",
        start_year=2025,
        ref_annual_improvement=1.2,
        annual_improvement=1.2,
        ei_improvement_rate=2.9, #could change to 1.7 or 1.8 not sure with documentation
        time_to_reach_long_term_rate=10
    )

    transport_sector = TechEffectModel(
        sector="transport",
        start_year=2025,
        ref_annual_improvement=1.2,
        annual_improvement=1.2,
        ei_improvement_rate=1.7,
        time_to_reach_long_term_rate=10
    )

    time_steps = range(2020, 2035)
    print(stationary_sector.simulate(time_steps))
    print(transport_sector.simulate(time_steps))

[(2020, 1), (2021, 1), (2022, 0.9413793103448276), (2023, 0.8886206896551724), (2024, 0.8411379310344828), (2025, 0.7984034482758621), (2026, 0.7599424137931035), (2027, 0.7253274827586207), (2028, 0.6941740448275862), (2029, 0.6661359506896551), (2030, 0.6409016659655172), (2031, 0.6181908097137931), (2032, 0.5977510390872414), (2033, 0.5793552455233448), (2034, 0.562799031315838)]
[(2020, 1), (2021, 1), (2022, 0.9705882352941176), (2023, 0.9441176470588235), (2024, 0.9202941176470588), (2025, 0.8988529411764706), (2026, 0.8795558823529412), (2027, 0.8621885294117648), (2028, 0.846557911764706), (2029, 0.832490355882353), (2030, 0.8198295555882353), (2031, 0.8084348353235294), (2032, 0.7981795870852941), (2033, 0.7889498636708824), (2034, 0.7806431125979117)]


Tech effect on energy intensity of new capital
UNITS
(dimensionless)

The below code prints stuff and does not provide a useful list like above but for now it works. a lot of the equations are the same so we can likely combine it all.



In [ ]:
class new_capital_tech_effect:
    def __init__(self, name, ei_improvement_rate, ref_annual_improvement):
        self.name = name
        self.ei_improvement_rate = ei_improvement_rate
        self.ref_annual_improvement = ref_annual_improvement
        self.tech_effect_rate_ratio = 1  # Initial value
        self.tech_effect_on_energy_intensity = 1  # Initial value
        self.start_year_intensity_improvement = 2025

    def rate_of_decrease_energy_intensity(self, time):
        return self.ei_improvement_rate * self.tech_effect_rate_ratio / 100

    def change_in_tech_effect_rate(self, time):
        if time <= 2021:
            return 0
        return self.tech_effect_rate_adjustment() * self.tech_effect_rate_ratio

    def tech_effect_rate_adjustment(self):
        return self.tech_effect_rate_gap() / 10  # Time to reach long-term intensity rate

    def tech_effect_rate_gap(self):
        if self.start_year_intensity_improvement > 2025:
            annual_improvement = self.ref_annual_improvement
        else:
            annual_improvement = self.ref_annual_improvement
        return (annual_improvement / self.rate_of_decrease_energy_intensity(2025)) - self.tech_effect_rate_ratio

    def decreasing_energy_requirements(self):
        min_value = min(
            self.tech_effect_on_energy_intensity * self.rate_of_decrease_energy_intensity(2025),
            max(0, self.tech_effect_on_energy_intensity / 1)  # One year = 1
        )
        return min_value

    def update_tech_effect(self):
        self.tech_effect_on_energy_intensity -= self.decreasing_energy_requirements()
        self.tech_effect_rate_ratio += self.change_in_tech_effect_rate(2025)

# Initialize sectors
residential = new_capital_tech_effect("Residential", 2.9, 1.2)
commercial = new_capital_tech_effect("Commercial", 1.7, 1.2)
industrial = new_capital_tech_effect("Industrial", 1.8, 0.5)

# Example simulation
for year in range(2025, 2035):
    print(f"Year: {year}")
    for sector in [residential, commercial, industrial]:
        sector.update_tech_effect()
        print(f"{sector.name} - New Capital Tech Effect on Energy Intensity: {sector.tech_effect_on_energy_intensity:.4f}")
    print("-")


Year: 2025
Residential - Tech Effect on Energy Intensity: 0.9710
Commercial - Tech Effect on Energy Intensity: 0.9830
Industrial - Tech Effect on Energy Intensity: 0.9820
-
Year: 2026
Residential - Tech Effect on Energy Intensity: 0.8291
Commercial - Tech Effect on Energy Intensity: 0.8500
Industrial - Tech Effect on Energy Intensity: 0.9170
-
Year: 2027
Residential - Tech Effect on Energy Intensity: 0.6695
Commercial - Tech Effect on Energy Intensity: 0.7245
Industrial - Tech Effect on Energy Intensity: 0.8328
-
Year: 2028
Residential - Tech Effect on Energy Intensity: 0.5459
Commercial - Tech Effect on Energy Intensity: 0.6235
Industrial - Tech Effect on Energy Intensity: 0.7537
-
Year: 2029
Residential - Tech Effect on Energy Intensity: 0.4437
Commercial - Tech Effect on Energy Intensity: 0.5331
Industrial - Tech Effect on Energy Intensity: 0.6822
-
Year: 2030
Residential - Tech Effect on Energy Intensity: 0.3610
Commercial - Tech Effect on Energy Intensity: 0.4577
Industrial - Tech

**Original energy requirements of capital 1**

For each end use and by carrier, the energy required of capital in vintage 1 as originally installed (prior to retrofits, if any).
UNITS
EJ/Year

Phase 1 Intial Effective Capital Age
This Python module models the energy-consuming capital dynamics across different sectors. It initializes capital levels based on GDP, computes desired capital, tracks capital aging and installation, and updates capital over time.

In [ ]:
import numpy as np

class EnergyConsumingCapital:
    def __init__(self, sector, gdp_by_sector, gdp_rate_of_change):
        self.sector = sector
        self.gdp_by_sector = gdp_by_sector
        self.gdp_rate_of_change = gdp_rate_of_change
        self.initial_capital_output_ratio = 2.5
        self.capital_lifetime = {"Residential": 30, "Commercial": 30, "Industrial": 15}[sector]
        self.time_to_adjust = 5  # Assumed time for capital adjustment

        self.initialize_capital()

    def initialize_capital(self):
        self.initial_energy_consuming_capital = self.gdp_by_sector * self.initial_capital_output_ratio
        self.vintage_1_lifetime = self.capital_lifetime / 3
        self.vintage_2_lifetime = self.capital_lifetime / 3
        self.vintage_3_lifetime = self.capital_lifetime / 3

        self.relative_initial_capital_v1 = np.exp(-self.gdp_rate_of_change / 100 * self.vintage_1_lifetime)
        self.relative_initial_capital_v2 = np.exp(-self.gdp_rate_of_change / 100 * (self.vintage_1_lifetime + self.vintage_2_lifetime))
        self.relative_initial_capital_v3 = 1 - (self.relative_initial_capital_v1 + self.relative_initial_capital_v2)

        sum_relative_capital = self.relative_initial_capital_v1 + self.relative_initial_capital_v2 + self.relative_initial_capital_v3

        self.frac_initial_capital_v1 = self.relative_initial_capital_v1 / sum_relative_capital
        self.frac_initial_capital_v2 = self.relative_initial_capital_v2 / sum_relative_capital
        self.frac_initial_capital_v3 = self.relative_initial_capital_v3 / sum_relative_capital

        self.energy_capital_1 = self.initial_energy_consuming_capital * self.frac_initial_capital_v1
        self.energy_capital_2 = self.initial_energy_consuming_capital * self.frac_initial_capital_v2
        self.energy_capital_3 = self.initial_energy_consuming_capital * self.frac_initial_capital_v3

        self.initial_effective_capital_age = (
            (self.energy_capital_1 * self.vintage_1_lifetime +
             self.energy_capital_2 * self.vintage_2_lifetime +
             self.energy_capital_3 * self.vintage_3_lifetime) /
            (self.energy_capital_1 + self.energy_capital_2 + self.energy_capital_3)
        )

    def compute_desired_capital(self):
        return self.gdp_by_sector * self.initial_capital_output_ratio

    def compute_total_energy_capital(self):
        return self.energy_capital_1 + self.energy_capital_2 + self.energy_capital_3

    def compute_installing_energy_capital(self):
        desired_capital = self.compute_desired_capital()
        total_capital = self.compute_total_energy_capital()
        return max(0, (desired_capital - total_capital) / self.time_to_adjust)

    def update_capital(self):
        installing_capital = self.compute_installing_energy_capital()
        aging_1_to_2 = self.energy_capital_1 / self.vintage_1_lifetime
        discarding_1 = self.energy_capital_1 / self.vintage_1_lifetime

        self.energy_capital_1 += installing_capital - aging_1_to_2 - discarding_1
        self.energy_capital_2 += aging_1_to_2 - (self.energy_capital_2 / self.vintage_2_lifetime)
        self.energy_capital_3 += (self.energy_capital_2 / self.vintage_2_lifetime) - (self.energy_capital_3 / self.vintage_3_lifetime)

    def compute_reference_energy_intensity(self, initial_energy_requirements, historic_rate_of_decrease):
        return (sum(initial_energy_requirements) / self.initial_energy_consuming_capital *
                np.exp(-historic_rate_of_decrease / 100 * self.initial_effective_capital_age))

    def __repr__(self):
        return (f"Sector: {self.sector}\n"
                f"Total Energy Capital: {self.compute_total_energy_capital():.2f}\n"
                f"Energy Capital 1: {self.energy_capital_1:.2f}, "
                f"Energy Capital 2: {self.energy_capital_2:.2f}, "
                f"Energy Capital 3: {self.energy_capital_3:.2f}\n")

# Example Usage
if __name__ == "__main__":
    sectors = ["Residential", "Commercial", "Industrial"]
    gdp_values = {"Residential": 1000, "Commercial": 2000, "Industrial": 3000}
    gdp_growth_rate = 2  # Example rate of change
    historic_rates = {"Residential": 0.9, "Commercial": 0.9, "Industrial": -0.5}
    initial_energy_requirements = {"Residential": [500], "Commercial": [1000], "Industrial": [1500]}

    capital_models = {sector: EnergyConsumingCapital(sector, gdp_values[sector], gdp_growth_rate) for sector in sectors}

    for _ in range(10):  # Simulating 10 time steps
        for sector, model in capital_models.items():
            model.update_capital()
            ref_energy_intensity = model.compute_reference_energy_intensity(initial_energy_requirements[sector], historic_rates[sector])
            print(model)
            print(f"Reference Energy Intensity for {sector}: {ref_energy_intensity:.2f}\n")


Sector: Residential
Total Energy Capital: 2421.29
Energy Capital 1: 1637.46, Energy Capital 2: 1712.90, Energy Capital 3: -929.07

Reference Energy Intensity for Residential: 0.18

Sector: Commercial
Total Energy Capital: 4842.58
Energy Capital 1: 3274.92, Energy Capital 2: 3425.81, Energy Capital 3: -1858.15

Reference Energy Intensity for Commercial: 0.18

Sector: Industrial
Total Energy Capital: 7253.93
Energy Capital 1: 4071.77, Energy Capital 2: 6269.64, Energy Capital 3: -3087.48

Reference Energy Intensity for Industrial: 0.21

Sector: Residential
Total Energy Capital: 2365.44
Energy Capital 1: 1325.71, Energy Capital 2: 1705.36, Energy Capital 3: -665.63

Reference Energy Intensity for Residential: 0.18

Sector: Commercial
Total Energy Capital: 4730.88
Energy Capital 1: 2651.42, Energy Capital 2: 3410.72, Energy Capital 3: -1331.26

Reference Energy Intensity for Commercial: 0.18

Sector: Industrial
Total Energy Capital: 7018.37
Energy Capital 1: 2492.28, Energy Capital 2: 5830